In [9]:
# load decoding metadata, multiple configuration outputs from multinews, and try
# to figure out why multinews performance doesn't improve with more articles


# add experiment which computes per-article rouge using inputs themselves as predicted summaries


In [10]:
from pathlib import Path
import json

import numpy as np

# Ghalandari 2020 evaluate
from transformer_decoding.evaluate import evaluate_rouge

In [11]:
DIR = Path('/home/chrishokamp/projects/dynamic-ensembles')

In [12]:
#evaluation_dataset = [json.loads(l) for l in open(DIR / 'data/downloads_from_MultiNews_gdrive/filtered_test.jsonl')]
evaluation_dataset = [json.loads(l) for l in open(DIR / 'data/downloads_from_MultiNews_gdrive/test.jsonl')]

# compare per-segment rouge scores for two different runs
# one_article_preds = [l.strip() for l in open(DIR / 'MultiNews_250_1_article_shuffled_study_metadata_eval_predicted_summaries.out')]
# two_article_preds = [l.strip() for l in open(DIR / 'MultiNews_250_2_articles_shuffled_study_metadata_eval_predicted_summaries.out')]

# one_article_preds = [l.strip() for l in open(DIR / 'MultiNews_250_1_article_shuffled_study_metadata_eval_predicted_summaries.out')]
# two_article_preds = [l.strip() for l in open(DIR / 'MultiNews_250_3_articles_shuffled_study_metadata_eval_predicted_summaries.out')]


one_article_preds = [l.strip() for l in open(DIR / 'MultiNews_1_article_eval_predicted_summaries.out')]

two_article_preds = [l.strip() for l in open(DIR / 'MultiNews_5_articles_eval_predicted_summaries.out')]
# two_article_preds = [l.strip() for l in open(DIR / 'MultiNews_5_articles_eval_predicted_summaries.out')]

gold_summaries = [r['summary'].strip() for r in evaluation_dataset]


# oracle 1: take the input article closest in length to the average summary, and use that as the summary
# avg_gold_length = np.mean([len(s) for s in gold_summaries])
# avg_gold_length = np.mean([len(s) for s in gold_summaries])


# oracle_1_preds = []
# for r in evaluation_dataset[:num_rows_to_eval]:
#     input_lens = [len(a['text']) for a in r['articles']]
# #     closest = np.argsort([l for l in input_lens])[0]
#     closest = np.argsort([abs(l - avg_gold_length) for l in input_lens])[0]
#     oracle_1_preds.append(r['articles'][closest]['text'])

# oracle_1_results, _ = evaluate_rouge(oracle_1_preds, gold_summaries)
# dodgy_idxs = set([idx for idx, value in enumerate(oracle_1_results['rouge-l']['f']) if value >= 0.5])
    
# one_article_preds = [p for idx, p in enumerate(one_article_preds) if idx not in dodgy_idxs]
# two_article_preds = [p for idx, p in enumerate(two_article_preds) if idx not in dodgy_idxs]
# gold_summaries = [s for idx, s in enumerate(gold_summaries) if idx not in dodgy_idxs]

one_article_results, _ = evaluate_rouge(one_article_preds, gold_summaries)
two_article_results, _ = evaluate_rouge(two_article_preds, gold_summaries)

In [13]:
# len(dodgy_idxs)
# avg_gold_length
# np.mean([len(p) for p in oracle_1_preds])

In [14]:
# oracle: closest to avg length of gold
# 0.19350096
# 0.22134864754098363
# 0.20888520491803278

# oracle: shortest in cluster
# 0.17646408000000002
# 0.21794454545454547
# 0.20803545454545455

# hypothesis: there's one good article and the rest are shit?
# in the case where finding the one good article will produce a better summary than any of the individual articles
#  by a large margin, 1-article can win by large amount, even when multi-article wins more overall

In [15]:
# np.mean(oracle_1_results['rouge-l']['f'])

NameError: name 'oracle_1_results' is not defined

In [16]:
np.mean(one_article_results['rouge-l']['f'])

0.22194032550693701

In [17]:
np.mean(two_article_results['rouge-l']['f'])

0.20399345250800424

In [18]:
len(two_article_results['rouge-l']['f'])

5622

In [ ]:
# one_article_results, _ = evaluate_rouge(one_article_preds[:10]+one_article_preds[11:], gold_summaries)
# two_article_results, _ = evaluate_rouge(two_article_preds[:10]+two_article_preds[11:], gold_summaries)

In [19]:
import numpy as np


comparison_metric = 'rouge-l'
# print(list(zip(one_article_results['rouge-l']['f'], two_article_results['rouge-l']['f'])))

win_balance = 0
s1_wins = []
s2_wins = []

win_margins = []


for r_idx, (s1, s2) in enumerate(zip(one_article_results['rouge-l']['f'], two_article_results['rouge-l']['f'])):
    if s1 > s2:
        print(f'{r_idx}: s1 better than s2 by {s1 - s2}, s1: {s1}, s2: {s2}')
        win_balance -= 1
        s1_wins.append(r_idx)
        win_margins.append(s1 - s2)
    else:
        print(f'{r_idx}: s2 better than s1 by {s2 - s1}, s1: {s1}, s2: {s2}')
        win_balance += 1
        s2_wins.append(r_idx)
        win_margins.append(s2 - s1)

win_margins = np.array(win_margins)

print(f'win balance: {win_balance}')

0: s2 better than s1 by 0.01446, s1: 0.23077, s2: 0.24523
1: s2 better than s1 by 0.025759999999999977, s1: 0.24281, s2: 0.26857
2: s2 better than s1 by 0.01820000000000002, s1: 0.20339, s2: 0.22159
3: s2 better than s1 by 0.03589999999999999, s1: 0.12648, s2: 0.16238
4: s1 better than s2 by 0.012200000000000016, s1: 0.17669, s2: 0.16449
5: s1 better than s2 by 0.05416999999999997, s1: 0.25986, s2: 0.20569
6: s1 better than s2 by 0.021580000000000016, s1: 0.1988, s2: 0.17722
7: s2 better than s1 by 0.05976000000000001, s1: 0.17259, s2: 0.23235
8: s1 better than s2 by 0.03130999999999999, s1: 0.15113, s2: 0.11982
9: s2 better than s1 by 0.016350000000000003, s1: 0.22608, s2: 0.24243
10: s1 better than s2 by 0.40426, s1: 0.70509, s2: 0.30083
11: s1 better than s2 by 0.015639999999999987, s1: 0.14354, s2: 0.1279
12: s1 better than s2 by 0.008989999999999998, s1: 0.2488, s2: 0.23981
13: s2 better than s1 by 0.0, s1: 0.21514, s2: 0.21514
14: s2 better than s1 by 0.023290000000000005, s1: 0.

2404: s1 better than s2 by 0.013769999999999977, s1: 0.17242, s2: 0.15865
2405: s1 better than s2 by 0.04360999999999998, s1: 0.23409, s2: 0.19048
2406: s2 better than s1 by 0.007640000000000008, s1: 0.21154, s2: 0.21918
2407: s1 better than s2 by 0.10240999999999997, s1: 0.27468, s2: 0.17227
2408: s2 better than s1 by 0.005619999999999986, s1: 0.21717, s2: 0.22279
2409: s2 better than s1 by 0.03159999999999999, s1: 0.1535, s2: 0.1851
2410: s1 better than s2 by 0.07522000000000001, s1: 0.28881, s2: 0.21359
2411: s1 better than s2 by 0.057609999999999995, s1: 0.2381, s2: 0.18049
2412: s2 better than s1 by 0.006869999999999987, s1: 0.1836, s2: 0.19047
2413: s2 better than s1 by 0.0018500000000000183, s1: 0.1838, s2: 0.18565
2414: s2 better than s1 by 0.06845, s1: 0.112, s2: 0.18045
2415: s1 better than s2 by 0.15019999999999997, s1: 0.35294, s2: 0.20274
2416: s2 better than s1 by 0.039199999999999985, s1: 0.15981, s2: 0.19901
2417: s2 better than s1 by 0.10059000000000001, s1: 0.14583, s

4036: s2 better than s1 by 0.010789999999999994, s1: 0.19444, s2: 0.20523
4037: s2 better than s1 by 0.007780000000000009, s1: 0.15814, s2: 0.16592
4038: s2 better than s1 by 0.005219999999999975, s1: 0.14444, s2: 0.14966
4039: s1 better than s2 by 0.017009999999999997, s1: 0.18261, s2: 0.1656
4040: s1 better than s2 by 0.07293, s1: 0.21764, s2: 0.14471
4041: s1 better than s2 by 0.009060000000000012, s1: 0.18182, s2: 0.17276
4042: s1 better than s2 by 0.08689999999999998, s1: 0.28362, s2: 0.19672
4043: s1 better than s2 by 0.012149999999999994, s1: 0.22407, s2: 0.21192
4044: s2 better than s1 by 0.11842000000000003, s1: 0.17062, s2: 0.28904
4045: s1 better than s2 by 0.028210000000000013, s1: 0.19123, s2: 0.16302
4046: s2 better than s1 by 0.029210000000000014, s1: 0.12893, s2: 0.15814
4047: s1 better than s2 by 0.16118999999999997, s1: 0.25275, s2: 0.09156
4048: s2 better than s1 by 0.0, s1: 0.21951, s2: 0.21951
4049: s1 better than s2 by 0.05399000000000001, s1: 0.22692, s2: 0.17293

In [20]:
s1_win_margins = win_margins[s1_wins]
s2_win_margins = win_margins[s2_wins]

In [26]:
print(s1_win_margins.mean())
print(s2_win_margins.mean())
print(s1_win_margins.std())
print(s2_win_margins.std())

0.06679801559177889
0.031288100000000006
0.08965733005443664
0.02874509987739525


In [23]:
win_balance

-22

In [ ]:
# MultiNews 1-article vs 5-articles
# print(s1_win_margins.mean())
# print(s2_win_margins.mean())
# print(s1_win_margins.std())
# print(s2_win_margins.std())
# 0.06679801559177889
# 0.031288100000000006
# 0.08965733005443664
# 0.02874509987739525

# win_balance
# -22


In [24]:
# 1-article win margins are way bigger, lets check a few items
biggest_margin_clusters = [s1_wins[idx] for idx in np.argsort(s1_win_margins)[::-1]][:5]

In [25]:
np.sort(s1_win_margins)[::-1][:10]

array([0.74316, 0.73102, 0.73045, 0.70803, 0.70348, 0.70152, 0.68586,
       0.68535, 0.67686, 0.67143])

In [84]:
all_eval_articles = []

import re
import Levenshtein
    

for c in evaluation_dataset:
    c_articles = []
    for a in c['articles']:
        a = ' '.join(re.sub('NEWLINE_CHAR', '', a['text']).split())
        c_articles.append(a)
    all_eval_articles.append(c_articles)

In [85]:
len(all_eval_articles)

5622

In [86]:
biggest_margin_clusters

[115, 160, 198, 10, 20]

In [87]:
evaluation_dataset[115]['articles'][1]

{'title': '',
 'text': 'SAN ANTONIO (AP) — Eight people have been found dead in a tractor-trailer outside a Walmart in Texas in what police are calling a horrific human trafficking case. NEWLINE_CHAR NEWLINE_CHAR Several other people, possibly dozens, were found in the truck, which didn\'t have a working air conditioning system, and they were taken to hospitals, firefighters said. NEWLINE_CHAR NEWLINE_CHAR A person from the truck approached a Walmart employee in a parking lot and asked for water late Saturday night or early Sunday morning, San Antonio police said. The employee gave the person the water and then called police, and when officers arrived they found the eight people dead in the back of the trailer, police Chief William McManus said. NEWLINE_CHAR NEWLINE_CHAR Investigators checked store surveillance video, which showed vehicles had arrived and picked up other people from the tractor-trailer, police said. NEWLINE_CHAR NEWLINE_CHAR "We\'re looking at a human trafficking crime

In [88]:
all_eval_articles[115][1]

'SAN ANTONIO (AP) — Eight people have been found dead in a tractor-trailer outside a Walmart in Texas in what police are calling a horrific human trafficking case. Several other people, possibly dozens, were found in the truck, which didn\'t have a working air conditioning system, and they were taken to hospitals, firefighters said. A person from the truck approached a Walmart employee in a parking lot and asked for water late Saturday night or early Sunday morning, San Antonio police said. The employee gave the person the water and then called police, and when officers arrived they found the eight people dead in the back of the trailer, police Chief William McManus said. Investigators checked store surveillance video, which showed vehicles had arrived and picked up other people from the tractor-trailer, police said. "We\'re looking at a human trafficking crime this evening," McManus said. He called the case "a horrific tragedy." The U.S. Department of Homeland Security was involved in

In [89]:
evaluation_dataset[115]['summary']

'– Eight people have been found dead in a tractor-trailer outside a Walmart in San Antonio, Texas, in what police are calling a horrific human trafficking case. Some 30 other people were found in the truck, which didn\'t have a working air conditioning system, and they were taken to hospitals, reports CNN. Seventeen of them are in critical condition, while 13 are in serious condition. Authorities said the driver had been held, but they didn\'t release the driver\'s identity. A person from the truck approached a Walmart employee in a parking lot and asked for water late Saturday or early Sunday, San Antonio police said. The employee gave the person the water and then called police, reports the AP, and when officers arrived they found the eight people dead in the back of the trailer, police Chief William McManus said. Investigators checked store surveillance video, which showed vehicles had arrived and picked up other people from the tractor-trailer, police said. "We\'re looking at a hum

In [90]:
Levenshtein.distance(evaluation_dataset[115]['summary'], all_eval_articles[115][1])

255

In [74]:
len(evaluation_dataset[115]['summary'])

1188

In [53]:
'dog' in 'fish dog'

True

In [77]:
len(set(evaluation_dataset[115]['summary'].split()).difference(set(all_eval_articles[115][1].split())))

26

In [76]:
len(evaluation_dataset[115]['summary'].split())

194

In [46]:
evaluation_dataset[115]['summary'][] in all_eval_articles[115][1]

False

In [51]:
import difflib

case_a = evaluation_dataset[115]['summary']
case_b = all_eval_articles[115][1]

output_list = [li for li in difflib.ndiff(case_a, case_b) if li[0] != ' ']


In [57]:
len(output_list)

1242

In [ ]:
import 

In [36]:
len(gold_summaries)

5622

In [ ]:
# TODO: BLEU between summaries and articles could be another good measure

In [91]:
import tqdm

tok_diff_threshold = 0.25

exact_substring_matches = 0
dodgy_idxs = []

for idx, (s, c_articles) in tqdm.tqdm(enumerate(zip(gold_summaries, all_eval_articles))):
    s_toks = set(s.split())
    for a in c_articles:
        diff = s_toks.difference(set(a.split()))
        if len(diff) / len(s_toks) < tok_diff_threshold:
            exact_substring_matches += 1
            print(f'{idx}: substring_match {exact_substring_matches}')
            dodgy_idxs.append(idx)

dodgy_idxs = set(dodgy_idxs)
print(exact_substring_matches)

622it [00:00, 3100.15it/s]

32: substring_match 1
41: substring_match 2
51: substring_match 3
115: substring_match 4
140: substring_match 5
151: substring_match 6
160: substring_match 7
198: substring_match 8
266: substring_match 9
302: substring_match 10
326: substring_match 11
362: substring_match 12
385: substring_match 13
388: substring_match 14
481: substring_match 15
500: substring_match 16
502: substring_match 17
514: substring_match 18
514: substring_match 19
516: substring_match 20
516: substring_match 21
542: substring_match 22
603: substring_match 23
630: substring_match 24
653: substring_match 25
657: substring_match 26


1270it [00:00, 3166.40it/s]

883: substring_match 27
884: substring_match 28
906: substring_match 29
928: substring_match 30
990: substring_match 31
1051: substring_match 32
1073: substring_match 33
1087: substring_match 34
1117: substring_match 35
1122: substring_match 36
1157: substring_match 37
1161: substring_match 38
1277: substring_match 39
1307: substring_match 40
1426: substring_match 41
1426: substring_match 42
1429: substring_match 43
1433: substring_match 44
1473: substring_match 45
1479: substring_match 46


1903it [00:00, 3172.00it/s]

...

3930: substring_match 117
3944: substring_match 118
4099: substring_match 119
4104: substring_match 120
4123: substring_match 121
4130: substring_match 122
4186: substring_match 123
4246: substring_match 124
4273: substring_match 125
4273: substring_match 126
4329: substring_match 127
4335: substring_match 128
4341: substring_match 129


5079it [00:01, 2944.14it/s]

4459: substring_match 130
4509: substring_match 131
4525: substring_match 132
4554: substring_match 133
4571: substring_match 134
4582: substring_match 135
4616: substring_match 136
4673: substring_match 137
4686: substring_match 138
4708: substring_match 139
4716: substring_match 140
4739: substring_match 141
4767: substring_match 142
4777: substring_match 143
4887: substring_match 144
4942: substring_match 145
4956: substring_match 146
4982: substring_match 147
5103: substring_match 148
5125: substring_match 149
5140: substring_match 150
5156: substring_match 151
5231: substring_match 152


5622it [00:01, 3091.03it/s]

5349: substring_match 153
5349: substring_match 154
5355: substring_match 155
5365: substring_match 156
5496: substring_match 157
5508: substring_match 158
5562: substring_match 159
5567: substring_match 160
160


In [93]:
one_article_preds_ = [p for idx, p in enumerate(one_article_preds) if idx not in dodgy_idxs]
two_article_preds_ = [p for idx, p in enumerate(two_article_preds) if idx not in dodgy_idxs]
gold_summaries_ = [s for idx, s in enumerate(gold_summaries) if idx not in dodgy_idxs]

one_article_results_, _ = evaluate_rouge(one_article_preds_, gold_summaries_)
two_article_results_, _ = evaluate_rouge(two_article_preds_, gold_summaries_)

In [95]:
np.mean(one_article_results_['rouge-l']['f'])

0.22044008264462808

In [97]:
np.mean(two_article_results_['rouge-l']['f'])

0.2109793388429752

In [98]:
import numpy as np


comparison_metric = 'rouge-l'
# print(list(zip(one_article_results['rouge-l']['f'], two_article_results['rouge-l']['f'])))

win_balance = 0
s1_wins = []
s2_wins = []

win_margins = []


for r_idx, (s1, s2) in enumerate(zip(one_article_results_['rouge-l']['f'], two_article_results_['rouge-l']['f'])):
    if s1 > s2:
        print(f'{r_idx}: s1 better than s2 by {s1 - s2}, s1: {s1}, s2: {s2}')
        win_balance -= 1
        s1_wins.append(r_idx)
        win_margins.append(s1 - s2)
    else:
        print(f'{r_idx}: s2 better than s1 by {s2 - s1}, s1: {s1}, s2: {s2}')
        win_balance += 1
        s2_wins.append(r_idx)
        win_margins.append(s2 - s1)

win_margins = np.array(win_margins)

print(f'win balance: {win_balance}')

0: s2 better than s1 by 0.013190000000000007, s1: 0.23204, s2: 0.24523
1: s2 better than s1 by 0.025759999999999977, s1: 0.24281, s2: 0.26857
2: s1 better than s2 by 3.0000000000002247e-05, s1: 0.22162, s2: 0.22159
3: s2 better than s1 by 0.022779999999999995, s1: 0.12648, s2: 0.14926
4: s1 better than s2 by 0.026520000000000016, s1: 0.19101, s2: 0.16449
5: s1 better than s2 by 0.05416999999999997, s1: 0.25986, s2: 0.20569
6: s1 better than s2 by 0.020700000000000024, s1: 0.19792, s2: 0.17722
7: s1 better than s2 by 0.07021, s1: 0.30256, s2: 0.23235
8: s2 better than s1 by 0.0, s1: 0.11764, s2: 0.11764
9: s1 better than s2 by 0.005629999999999996, s1: 0.24806, s2: 0.24243
10: s1 better than s2 by 0.40426, s1: 0.70509, s2: 0.30083
11: s1 better than s2 by 0.015639999999999987, s1: 0.14354, s2: 0.1279
12: s2 better than s1 by 0.02503999999999998, s1: 0.2488, s2: 0.27384
13: s2 better than s1 by 0.0, s1: 0.21514, s2: 0.21514
14: s2 better than s1 by 0.023290000000000005, s1: 0.22727, s2: 

In [99]:
len(s2_wins)

131

In [100]:
len(s1_wins)

111

In [102]:
s1_wins

[2,
 4,
 5,
 6,
 7,
 9,
 10,
 11,
 16,
 18,
 19,
 20,
 28,
 31,
 33,
 36,
 40,
 42,
 43,
 44,
 45,
 47,
 48,
 49,
 51,
 54,
 55,
 56,
 57,
 62,
 63,
 64,
 66,
 67,
 69,
 71,
 74,
 75,
 76,
 79,
 80,
 82,
 83,
 87,
 88,
 89,
 91,
 95,
 96,
 98,
 99,
 101,
 104,
 107,
 110,
 111,
 112,
 113,
 114,
 117,
 119,
 123,
 125,
 127,
 130,
 134,
 135,
 136,
 137,
 138,
 139,
 141,
 142,
 143,
 144,
 148,
 149,
 153,
 154,
 164,
 166,
 168,
 172,
 175,
 184,
 187,
 188,
 190,
 191,
 193,
 195,
 196,
 197,
 200,
 201,
 203,
 206,
 207,
 211,
 212,
 213,
 214,
 216,
 223,
 228,
 229,
 230,
 231,
 234,
 238,
 241]

In [103]:
np.argsort(np.array(win_margins)[s1_wins])[::-1]

array([  6,  11,  45,  58,  77,  95,  27,  48,  96,  32,  79,  83,  47,
        17,  30,  35,  82,  78,  73, 106,  24,  54,  56, 107,  72,  81,
       110,  86,   4, 102,  12,  40,  87,  90,  51,  29,  22,  92,  74,
         2,  34,  13,  36,  88,  50,  61,  71,  75,  10,  49,  26,  99,
        65,  66, 104,  43,   9, 108,  25, 109,  41,  62,  94,  68,  97,
        69,  53, 100,  37,   1,  98,  31, 101,  85,  38,  14,  59,  57,
        55,   3,  15,  20,  93, 105,  46,  28,   7,  60, 103,  44,  52,
        70,  63,  84,  19,  21,  39,  80,  33,  23,  18,  42,  89,   5,
        67,  91,   8,  16,  64,  76,   0])

In [104]:
one_article_preds_[6]

'– Israel and Hamas traded accusations of a new round of rocket fire today, with the latter accusing the other side of "a formula of a one-sided aggression by the occupation on flimsy pretexts," reports Reuters. Israel says it hit targets in the Gaza Strip after Palestinian militants fired rockets at southern Israel, in what they said was a response to an Israeli air strike that killed one militant and wounded a second a day earlier. The Israeli army says it targeted "Hamas terror activity sites and terrorist squads responsible for the rocket fire," but gave no details. The armed wing of Hamas, the Islamists who control Gaza, said it had joined in today\'s rocket attack along with the smaller Islamic Jihad group, the first time since June that Hamas had acknowledged launching rockets at Israel. A Hamas spokesman said the movement would not remain passive in the face of what it called "one-sided" Israeli violence. "Should the enemy continue its aggression ... the reaction by the resista

In [106]:
two_article_preds_[6]

'– The Israeli military says it shot down an unmanned drone that crossed into its airspace over the weekend, but it\'s not clear whether it was an Iranian drone or one of its own. The drone was spotted flying over the Mediterranean Sea near the Gaza Strip, reports Reuters, and Israel says it was shot down by an Israeli warplane, though it\'s unclear if it was actually a drone or a drone launched by Iran. The Israeli air force says the drone was fired on from a military base in the West Bank, and the Israeli military is investigating whether it came from Iran, reports the Jerusalem Post. Iran, however, says it wasn\'t, and a spokesman for the country\'s Islamic Revolutionary Guard Corps says it\'s a drone that was launched from Iran\'s territory, not Israel\'s territory. "We don’t know where the drone came from, but we know that it was not from Iran," he said. "It was a drone, and it was launched by a terrorist group, and we don\'t know who it was from." Israel has not confirmed the dro

In [107]:
gold_summaries_[6]

'– Israel launched a round of airstrikes on Gaza today, wounding two militants and eight bystanders, including some children. Israel says the attack was retaliation for a round of Palestinian rockets fired yesterday, which Palestinians say were in response to another Israeli airstrike earlier in the day, Reuters reports. More than 470 rockets have been fired from Gaza since June, but this was the first time Hamas acknowledged responsibility. As usual the rockets failed to harm anyone, though one did kill some goats at a petting zoo. The incident comes in the wake of a drone flight into Israeli airspace this weekend, which some have speculated came from Iran. An Iranian military official today boasted that the incident proved that Israel is vulnerable, and that its anti-missile defense system "does not work and lacks the necessary capacity." But he denied that the drone was made by Iran, saying such accusations represented a "psychological operation" on Israel\'s part. "The Zionist regi

In [105]:
evaluation_dataset[6]

{'articles': [{'title': '',
   'text': 'A still image taken from Israeli Defence Forces (IDF) video footage shows what they say is a small unidentified aircraft shot down in a mid-air interception after it crossed into southern Israel October 6, 2012. NEWLINE_CHAR NEWLINE_CHAR DUBAI (Reuters) - The incursion by an unmanned aircraft into Israeli airspace at the weekend exposed the weakness of Israeli air defenses, an Iranian military official was quoted as saying on Monday. NEWLINE_CHAR NEWLINE_CHAR The Israeli air force shot down a drone on Saturday after it crossed into southern Israel, the military said, but it remained unclear where the aircraft had come from. NEWLINE_CHAR NEWLINE_CHAR Jamaluddin Aberoumand, deputy coordinator for Iran\'s Islamic Revolutionary Guard Corps, said the incident indicated that Israel\'s Iron Dome anti-missile defense system "does not work and lacks the necessary capacity", Fars news agency reported. NEWLINE_CHAR NEWLINE_CHAR The Iron Dome system, jointly

In [27]:
for c_idx in biggest_margin_clusters:
    print(json.dumps(evaluation_dataset[c_idx], indent=2))

{
  "articles": [
    {
      "title": "",
      "text": "(CNN) At least nine people are dead after a phone call from a Walmart employee led to the discovery of dozens of undocumented immigrants severely injured in the back of a tractor-trailer in Texas early Sunday, according to federal authorities. NEWLINE_CHAR NEWLINE_CHAR More than 100 undocumented immigrants may have been sandwiched in the back of the 18-wheeler during the journey, US Immigration and Customs Enforcement acting Director Thomas Homan said, citing initial interviews with survivors. NEWLINE_CHAR NEWLINE_CHAR \"Checking the video from the store, we found there were a number of vehicles that came in and picked up a lot of the folks that were in that trailer that survived the trip,\" San Antonio Police Chief William McManus said. \"We are looking at a human trafficking crime here.\" NEWLINE_CHAR NEWLINE_CHAR The employee told authorities the trailer was parked at a Walmart lot in San Antonio. A man who was in it asked th

In [23]:
biggest_margin_clusters

[115, 160, 198, 10, 20]

In [ ]:
one_article_preds[115]

In [ ]:
two_article_preds[115]

In [ ]:
evaluation_dataset[115]

In [ ]:
list(np.sort(s1_win_margins)[::-1])

In [ ]:
list(np.sort(s2_win_margins)[::-1])

In [ ]:
win_balance

In [ ]:
import numpy

In [ ]:
print(json.dumps(list(zip(one_article_preds, two_article_preds)), indent=2))